#  Detección de placas de automóviles en imágenes

Este trabajo aborda la detección automática de placas vehiculares en imágenes mediante tres variantes de la familia YOLO (YOLOv11-n, YOLOv11-s y YOLOv10-s), seleccionadas por su bajo costo computacional y reducido número de parámetros.

El trabajo se realizo en varios Notebook usando python, por lo que en este se unieron todos los notebooks y se explica el flujo de trabajo.

##### Consideraciones

+ En caso de ejecutar crear el entorno en conda propuesto en la parte de abajo
+  Instalar los siguientes modelos, ver en: (https://docs.ultralytics.com/es/models/)
      +  yolov10s.pt
      +  yolo11s.pt

In [ ]:
# Entorno recmenddo para ejecutar este notebook
conda create -n yolov8_env python=3.10 -y
conda activate yolov8_env
#En caso de tener GPU
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
#En caso de solo tener CPU
pip install torch torchvision torchaudio
#Otras dependencias
pip install ultralytics
pip install pandas scikit-learn opencv-python-headless pyyaml matplotlib
pip install "numpy>=1.24,<2.0"
pip install pyarrow --no-build-isolation

In [1]:
import pandas as pd 
import yaml
from pathlib import Path
from itertools import product
from sklearn.model_selection import KFold, train_test_split
from ultralytics import YOLO
import os

### Validacion cruzada

El propósito de los siguientes chunks es preparar los datos para validacion cruzada (K-Fold) aplicado en la seleccion de hiperparametros, para ello solo tomamos 10%  de los datos (Dado el coste computacional) en training y test,  posteriormente evaluamos y obtenemos los mejores.


In [ ]:
ROOT = Path(r"/Users/pepefv97/Downloads/DATOS_PLACAS/train")  # raíz original (!Modificar en caso de correr el codigo)
IMG_DIR = ROOT / "images"
LBL_DIR = ROOT / "labels"


imgs = sorted(IMG_DIR.rglob("*.jpg"))
df   = pd.DataFrame({"img": imgs})
train_df = df.sample(frac=0.10, random_state=123).reset_index(drop=True)


K = 5
kf = KFold(n_splits=K, shuffle=True, random_state=123)

In [ ]:
OUT_ROOT = Path("folds")                     # carpeta raíz donde viven todos los folds
OUT_ROOT.mkdir(exist_ok=True)

fold_paths = []

for fold_i, (tr, va) in enumerate(kf.split(train_df)):
    tr_imgs = train_df.iloc[tr]["img"].tolist()
    va_imgs = train_df.iloc[va]["img"].tolist()

    # --- 1. Crear la estructura de carpetas -------------------------
    fold_dir = OUT_ROOT / f"fold{fold_i}"
    for split in ["train", "val"]:
        for sub in ["images", "labels"]:
            (fold_dir / split / sub).mkdir(parents=True, exist_ok=True)

    # --- 2. Hard‑links para imágenes y labels -----------------------
    def link(img_path, dst_img_dir):
        src_img = Path(img_path)
        src_lbl = LBL_DIR / src_img.with_suffix(".txt").name
        dst_img = dst_img_dir / src_img.name
        dst_lbl = dst_img_dir.parent / "labels" / src_lbl.name
        os.link(src_img, dst_img)   # crea enlace duro a la imagen
        os.link(src_lbl, dst_lbl)   # crea enlace duro al .txt

    for p in tr_imgs:
        link(p, fold_dir / "train" / "images")
    for p in va_imgs:
        link(p, fold_dir / "val" / "images")

    # --- 3. YAML: usa rutas directas de este fold (¡sin duplicar!) --
    yaml.dump(
        {
            "train": str((fold_dir / "train" / "images").resolve()),
            "val":   str((fold_dir / "val"   / "images").resolve()),
            "nc": 1,
            "names": ["PLACA_AUTOMOVIL"]
        },
        open(fold_dir / "data.yaml", "w")
    )

    fold_paths.append(fold_dir / "data.yaml")   # guardar ruta para entrenos posteriores

In [ ]:
param_grid = {
    "optimizer": ["SGD", "AdamW"],             # mejores en detección :contentReference[oaicite:3]{index=3}
    "lr0": [0.001, 0.005],
    "weight_decay": [1e-4, 5e-4],
    "mosaic": [0.5, 1.0]
}
grid = [dict(zip(param_grid, v)) for v in product(*param_grid.values())]

params_train = dict(epochs=3, batch=32, imgsz=640,
              device="mps", freeze=10, plots=True, verbose=True)

In [ ]:
for cfg in grid:
    for fold_yaml in fold_paths:
        name = (
            f"{Path(fold_yaml).parent.name}"
            f"_opt-{cfg['optimizer']}"
            f"_lr{cfg['lr0']}"
            f"_wd{cfg['weight_decay']}"
            f"_mos{cfg['mosaic']}"
        )
        YOLO("yolo11n.pt").train(
            data=str(fold_yaml),
            project="cross_validation_hpt",
            name=name,
            **cfg,
            **params_train
        )

Observando las metricas de nuestro modelo:

In [ ]:
# -------------------------------------------------------------
# LEER TODOS LOS metrics.csv Y AGREGAR POR COMBINACIÓN
# -------------------------------------------------------------
import glob, re, pandas as pd
from pathlib import Path

ROOT_RUNS = Path("cross_validation_hpt")   # carpeta raíz de tus runs
pattern   = re.compile(
    r"fold\d+_"            # foldX_
    r"opt-(?P<opt>\w+)_"
    r"lr(?P<lr>[\d.]+)_"
    r"wd(?P<wd>[\d.]+)_"
    r"mos(?P<mos>[\d.]+)"
)  # ← extrae hiperparámetros del nombre de carpeta

records = []

for csv in ROOT_RUNS.rglob("results.csv"):
    run_dir = csv.parent
    m = pattern.search(run_dir.name)
    if not m:
        continue  # ignora carpetas que no sigan el patrón
    hp = m.groupdict()                 # dict con opt, lr, wd, mos
    df = pd.read_csv(csv)
    best_map50 = df["metrics/mAP50(B)"].max()     # mejor época
    fold       = run_dir.name.split("_")[0]     # fold0, fold1…
    records.append({**hp, "fold": fold, "mAP50": best_map50})

# -------------------------------------------------------------
# PROMEDIAR Y CALCULAR (media ± std) POR COMBINACIÓN
# -------------------------------------------------------------
df = pd.DataFrame(records)
group_cols = ["opt", "lr", "wd", "mos"]
summary = (
    df.groupby(group_cols)["mAP50"]
      .agg(["mean", "std", "count"])
      .reset_index()
      .sort_values("mean", ascending=False)
)
print(summary.head(5))

Entrenando nuestro modelo con los mejores hiperparametros:

In [ ]:
from ultralytics import YOLO

path_yml = r'/Users/pepefv97/Downloads/DATOS_PLACAS/data.yaml'


model = YOLO("yolo11n.pt")      
model.train(
    data=path_yml,      
    epochs=10,                
    batch=32,
    imgsz=640,
    device="mps",
    optimizer="AdamW",
    lr0=0.001,
    weight_decay=0.0001,
    mosaic=1.0,                
    freeze=10,                  
    plots=True,
    project="ENTRENAMIENTO_YOLOV11",
    name="YOLOV11_AdamW_lr0.001_wd1e-4_mos1.0",
    verbose=True
)

In [ ]:
# -------------------------------------------------------------
# LEER TODOS LOS metrics.csv Y AGREGAR POR COMBINACIÓN
# -------------------------------------------------------------
import glob, re, pandas as pd
from pathlib import Path

ROOT_RUNS = Path("yolov11_cross_validation_hpt")
pattern   = re.compile(
    r"fold\d+_"                # foldX_
    r"opt-(?P<opt>\w+)_"
    r"lr(?P<lr>[\d.]+)_"
    r"wd(?P<wd>[\d.]+)_"
    r"mos(?P<mos>[\d.]+)"
)

records = []

for csv in ROOT_RUNS.rglob("results.csv"):
    run_dir = csv.parent
    m = pattern.search(run_dir.name)
    if not m:
        continue                                           # ignora carpetas mal nombradas
    hp = m.groupdict()                                     # hiperparámetros
    df = pd.read_csv(csv)

    # --- localizar la FILA con el mejor mAP50 ---
    best_row        = df.loc[df["metrics/mAP50(B)"].idxmax()]
    best_map50      = best_row["metrics/mAP50(B)"]
    best_map5095    = best_row["metrics/mAP50-95(B)"]      # <- misma época

    records.append({
        **hp,
        "fold"     : run_dir.name.split("_")[0],           # fold0, fold1…
        "mAP50"    : best_map50,
        "mAP50_95" : best_map5095
    })

# -------------------------------------------------------------
# PROMEDIAR (media ± std) POR COMBINACIÓN
# -------------------------------------------------------------
df = pd.DataFrame(records)
group_cols = ["opt", "lr", "wd", "mos"]

summary = (
    df.groupby(group_cols)
      .agg(
          mean    = ("mAP50",    "mean"),   # igual que antes
          std     = ("mAP50",    "std"),
          count   = ("fold",     "count"),
          mean95  = ("mAP50_95", "mean"),   # nuevas columnas
          std95   = ("mAP50_95", "std")
      )
      .reset_index()
      .sort_values("mean", ascending=False)  # mismo criterio de orden
)

print(summary.head(5))


In [ ]:
# -------------------------------------------------------------
# LEER TODOS LOS metrics.csv Y AGREGAR POR COMBINACIÓN
# -------------------------------------------------------------
import glob, re, pandas as pd
from pathlib import Path

ROOT_RUNS = Path("yolov10_cross_validation_hpt")
pattern   = re.compile(
    r"fold\d+_"                # foldX_
    r"opt-(?P<opt>\w+)_"
    r"lr(?P<lr>[\d.]+)_"
    r"wd(?P<wd>[\d.]+)_"
    r"mos(?P<mos>[\d.]+)"
)

records = []

for csv in ROOT_RUNS.rglob("results.csv"):
    run_dir = csv.parent
    m = pattern.search(run_dir.name)
    if not m:
        continue                                           # ignora carpetas mal nombradas
    hp = m.groupdict()                                     # hiperparámetros
    df = pd.read_csv(csv)

    # --- localizar la FILA con el mejor mAP50 ---
    best_row        = df.loc[df["metrics/mAP50(B)"].idxmax()]
    best_map50      = best_row["metrics/mAP50(B)"]
    best_map5095    = best_row["metrics/mAP50-95(B)"]      # <- misma época

    records.append({
        **hp,
        "fold"     : run_dir.name.split("_")[0],           # fold0, fold1…
        "mAP50"    : best_map50,
        "mAP50_95" : best_map5095
    })

# -------------------------------------------------------------
# PROMEDIAR (media ± std) POR COMBINACIÓN
# -------------------------------------------------------------
df = pd.DataFrame(records)
group_cols = ["opt", "lr", "wd", "mos"]

summary = (
    df.groupby(group_cols)
      .agg(
          mean    = ("mAP50",    "mean"),   # igual que antes
          std     = ("mAP50",    "std"),
          count   = ("fold",     "count"),
          mean95  = ("mAP50_95", "mean"),   # nuevas columnas
          std95   = ("mAP50_95", "std")
      )
      .reset_index()
      .sort_values("mean", ascending=False)  # mismo criterio de orden
)

print(summary.head(5))

--------------------------------------------

In [ ]:
param_grid = {
    "optimizer":     ["SGD", "AdamW"],
    "lr0":           [1e-3, 5e-4], # Tasa de aprendizaje de nuestro modelo
    "weight_decay":  [1e-4, 5e-4], # Tasa de decaimiento, para evitar el sobreajuste
    "mosaic":        [0,0.5]
}

grid = [dict(zip(param_grid, v)) for v in product(*param_grid.values())]

params_train = dict(
    epochs=3, # Numero de epocas a entrenar el modelo
    batch=32, # Tamaño del batch (Cuantas imagenes se entrenan al mismo tiempo)
    imgsz=512, # Tamaño de las imagenes a entrenar
    device="mps", # Dispositvo a usar, ("mps" para Mac con chip M1/M2, "cuda" para GPU Nvidia, "cpu" para CPU) (!En cado de correr el modelo cambiar la configuración del dispositivo!)
    freeze=10, # Congelar las primeras 10 capas del modelo (Se aplica en transfer learning)
    plots=True, # Mostrar las graficas de entrenamiento
    verbose=True, # Muestra info mas detallada del entrenamiento
    amp=True # Usar precision mixta (Mejora el rendimiento en GPU)
)

fold_paths = ['folds/fold0/data.yaml','folds/fold1/data.yaml','folds/fold2/data.yaml','folds/fold3/data.yaml','folds/fold4/data.yaml'] #Nombres de las carpetas de los folds



### Ejecutando con nuestro modelo yolov10s.pt

In [ ]:
# Generando un folder con  nombre para cada cada experimento especificando los hiperparametros
for cfg in grid:
    for fold_yaml in fold_paths:
        name = (
            f"{Path(fold_yaml).parent.name}"
            f"_opt-{cfg['optimizer']}"
            f"_lr{cfg['lr0']}"
            f"_wd{cfg['weight_decay']}"
            f"_mos{cfg['mosaic']}"
        )
        YOLO("yolov10s.pt").train( # Entremaneiento con el modelo YOLOv10s
            data=str(fold_yaml),
            project="yolov10_cross_validation_hpt",
            name=name,
            **cfg,
            **params_train
        )

Codigo para ir leyendo los resultados que nos dio nuestros modelos

In [ ]:
# -------------------------------------------------------------
# LEER TODOS LOS metrics.csv Y AGREGAR POR COMBINACIÓN
# -------------------------------------------------------------
import glob, re, pandas as pd
from pathlib import Path

ROOT_RUNS = Path("yolov10_cross_validation_hpt")   # carpeta raíz de tus runs
pattern   = re.compile(
    r"fold\d+_"            # foldX_
    r"opt-(?P<opt>\w+)_"
    r"lr(?P<lr>[\d.]+)_"
    r"wd(?P<wd>[\d.]+)_"
    r"mos(?P<mos>[\d.]+)"
)  # ← extrae hiperparámetros del nombre de carpeta

records = []

for csv in ROOT_RUNS.rglob("results.csv"):
    run_dir = csv.parent
    m = pattern.search(run_dir.name)
    if not m:
        continue  # ignora carpetas que no sigan el patrón
    hp = m.groupdict()                 # dict con opt, lr, wd, mos
    df = pd.read_csv(csv)
    best_map50 = df["metrics/mAP50(B)"].max()     # mejor época
    fold       = run_dir.name.split("_")[0]     # fold0, fold1…
    records.append({**hp, "fold": fold, "mAP50": best_map50})

# -------------------------------------------------------------
# PROMEDIAR Y CALCULAR (media ± std) POR COMBINACIÓN
# -------------------------------------------------------------
df = pd.DataFrame(records)
group_cols = ["opt", "lr", "wd", "mos"]
summary = (
    df.groupby(group_cols)["mAP50"]
      .agg(["mean", "std", "count"])
      .reset_index()
      .sort_values("mean", ascending=False)
)
print(summary.head(10))

Ejecutando el modelo con los mejores hiperparametros encontrados en el chunk anterior

In [ ]:
from ultralytics import YOLO

path_yml = r'/Users/pepefv97/Downloads/DATOS_PLACAS/data.yaml'


model = YOLO("yolov10s.pt")      
model.train(
    data=path_yml, 
    epochs=10,
    batch=32,
    imgsz=512,
    device="mps",
    freeze=10,
    plots=True,
    verbose=True,
    amp=True,
    optimizer="AdamW",
    lr0=0.0005,
    weight_decay=0.0005,
    mosaic=0,                
    project="ENTRENAMIENTO_YOLOV10S",
    name="YOLOV10S_AdamW_lr0.0005_wd0.0005_mos0"
)

### Ejecutando con nuestro modelo yolo11s.pt

In [ ]:
for cfg in grid:
    for fold_yaml in fold_paths:
        name = (
            f"{Path(fold_yaml).parent.name}"
            f"_opt-{cfg['optimizer']}"
            f"_lr{cfg['lr0']}"
            f"_wd{cfg['weight_decay']}"
            f"_mos{cfg['mosaic']}"
        )
        YOLO("yolo11s.pt").train(
            data=str(fold_yaml),
            project="yolov11_cross_validation_hpt",
            name=name,
            **cfg,
            **params_train
        )

Codigo para ir leyendo los resultados que nos dio nuestros modelos

In [ ]:
# -------------------------------------------------------------
# LEER TODOS LOS metrics.csv Y AGREGAR POR COMBINACIÓN
# -------------------------------------------------------------
import glob, re, pandas as pd
from pathlib import Path

ROOT_RUNS = Path("yolov11_cross_validation_hpt")   # carpeta raíz de tus runs
pattern   = re.compile(
    r"fold\d+_"            # foldX_
    r"opt-(?P<opt>\w+)_"
    r"lr(?P<lr>[\d.]+)_"
    r"wd(?P<wd>[\d.]+)_"
    r"mos(?P<mos>[\d.]+)"
)  # ← extrae hiperparámetros del nombre de carpeta

records = []

for csv in ROOT_RUNS.rglob("results.csv"):
    run_dir = csv.parent
    m = pattern.search(run_dir.name)
    if not m:
        continue  # ignora carpetas que no sigan el patrón
    hp = m.groupdict()                 # dict con opt, lr, wd, mos
    df = pd.read_csv(csv)
    best_map50 = df["metrics/mAP50(B)"].max()     # mejor época
    fold       = run_dir.name.split("_")[0]     # fold0, fold1…
    records.append({**hp, "fold": fold, "mAP50": best_map50})

# -------------------------------------------------------------
# PROMEDIAR Y CALCULAR (media ± std) POR COMBINACIÓN
# -------------------------------------------------------------
df = pd.DataFrame(records)
group_cols = ["opt", "lr", "wd", "mos"]
summary = (
    df.groupby(group_cols)["mAP50"]
      .agg(["mean", "std", "count"])
      .reset_index()
      .sort_values("mean", ascending=False)
)
print(summary.head(10))

In [ ]:
#Ejecuentando el modelo con los mejores hiperparametros encontrados

path_yml = r'/Users/pepefv97/Downloads/DATOS_PLACAS/data.yaml'


model = YOLO("yolo11s.pt")      
model.train(
    data=path_yml, 
    epochs=10,
    batch=32,
    imgsz=512,
    device="mps",
    freeze=10,
    plots=True,
    verbose=True,
    amp=True,
    optimizer="AdamW",
    lr0=0.0005,
    weight_decay=0.0005,
    mosaic=0,                
    project="ENTRENAMIENTO_YOLOV11S",
    name="YOLOV11S_AdamW_lr0.0005_wd0.0005_mos0"
)

### Evaluacion del modelo usando Boostrap

En este notebook se va a realizar el código para la evaluacion de los tres modelos entrenados. Para esto se sacarán las métricas $mAP50$ y $mAP50-95$ directamente sobre el conjunto de entrenamiento, así como generar intervalos de confianza para estas utilizando el método de Bootstrap.

In [ ]:
from ultralytics import YOLO
import pandas as pd

In [ ]:
model_v11n = YOLO('ENTRENAMIENTO_YOLOV11/YOLOV11_AdamW_lr0.001_wd1e-4_mos1.0/weights/best.pt')
model_v11s = YOLO('ENTRENAMIENTO_YOLOV11S/YOLOV11S_AdamW_lr0.0005_wd0.0005_mos0/weights/best.pt')
model_v10s = YOLO('ENTRENAMIENTO_YOLOV10S/YOLOV10S_AdamW_lr0.0005_wd0.0005_mos0/weights/best.pt')

In [ ]:
# Parametros para los modelos
metrics_yolov11n = model_v11n.val(
    data=r'/Users/pepefv97/Downloads/DATOS_PLACAS/data.yaml',
    split='test',
    workers=0,      
    verbose=False,
    save=False,
    plots=False,
    project="EVALUACION",
    name="YOLOV11N",
    device="mps",
    batch=64
)

metrics_yolov11s = model_v11s.val(
    data=r'/Users/pepefv97/Downloads/DATOS_PLACAS/data.yaml',
    split='test',
    workers=0,      
    verbose=False,
    save=False,
    plots=False,
    project="EVALUACION",
    name="YOLOV11S",
    device="mps",
    batch=64
)

metrics_yolov10s = model_v10s.val(
    data=r'/Users/pepefv97/Downloads/DATOS_PLACAS/data.yaml',
    split='test',
    workers=0,      
    verbose=False,
    save=False,
    plots=False,
    project="EVALUACION",
    name="YOLOV10S",
    device="mps",
    batch=64
)


In [ ]:
# Metricas para los modelos
metricas = {
    'Precision': metrics_yolov11n.box.mp,
    'Recall': metrics_yolov11n.box.mr,
    'mAP@0.5': metrics_yolov11n.box.map50,
    'mAP@0.5:0.95': metrics_yolov11n.box.map,
    'Tiempo Preprocesamiento (ms/img)': metrics_yolov11n.speed['preprocess'],
    'Tiempo Inferencia (ms/img)': metrics_yolov11n.speed['inference'],
    'Tiempo Postprocesamiento (ms/img)': metrics_yolov11n.speed['postprocess'],
    'Número de clases detectadas': metrics_yolov11n.box.nc
}

df_metrics = pd.DataFrame([metricas])
df_metrics.to_csv("EVALUACION/YOLOV11N/metrics_test.csv", index=False)


metricas_10s = {
    'Precision': metrics_yolov10s.box.mp,
    'Recall': metrics_yolov10s.box.mr,
    'mAP@0.5': metrics_yolov10s.box.map50,
    'mAP@0.5:0.95': metrics_yolov10s.box.map,
    'Tiempo Preprocesamiento (ms/img)': metrics_yolov10s.speed['preprocess'],
    'Tiempo Inferencia (ms/img)': metrics_yolov10s.speed['inference'],
    'Tiempo Postprocesamiento (ms/img)': metrics_yolov10s.speed['postprocess'],
    'Número de clases detectadas': metrics_yolov10s.box.nc
}

df_metrics_10s = pd.DataFrame([metricas_10s])
df_metrics_10s.to_csv("EVALUACION/YOLOV10S/metrics_test.csv", index=False)


metricas_11s = {
    'Precision': metrics_yolov11s.box.mp,
    'Recall': metrics_yolov11s.box.mr,
    'mAP@0.5': metrics_yolov11s.box.map50,
    'mAP@0.5:0.95': metrics_yolov11s.box.map,
    'Tiempo Preprocesamiento (ms/img)': metrics_yolov11s.speed['preprocess'],
    'Tiempo Inferencia (ms/img)': metrics_yolov11s.speed['inference'],
    'Tiempo Postprocesamiento (ms/img)': metrics_yolov11s.speed['postprocess'],
    'Número de clases detectadas': metrics_yolov11s.box.nc
}

df_metrics_11s = pd.DataFrame([metricas_11s])
df_metrics_11s.to_csv("EVALUACION/YOLOV11S/metrics_test.csv", index=False)



**Generación de intervalos de confianza para las métricas.**

In [ ]:
import random, pathlib, os, shutil, yaml, uuid

B  = 1000                              
TEST_IMG = pathlib.Path("/Users/pepefv97/Downloads/DATOS_PLACAS/test/images")   
TEST_LAB = pathlib.Path("/Users/pepefv97/Downloads/DATOS_PLACAS/test/labels")   
BOOT_ROOT = pathlib.Path("BOOTSTRAP/SETS")
CLASS_NAME = ["PLACA_AUTOMOVIL"] 
ROOT_PATH = pathlib.Path("/Users/pepefv97/Downloads/PROYECTO_5")
imgs = sorted(TEST_IMG.glob("*.jpg")) 

In [ ]:
random.seed(123)                               

for b in range(B):
    set_dir  = ROOT_PATH / BOOT_ROOT / f"set_{b:04d}"
    img_dir  = ROOT_PATH / set_dir / "test/images"
    lab_dir  = ROOT_PATH / set_dir / "test/labels"
    img_dir.mkdir(parents=True, exist_ok=True)
    lab_dir.mkdir(parents=True, exist_ok=True)

    sample = random.choices(imgs, k=len(imgs))  
    for idx, src in enumerate(sample):
        tag      = f"{idx:04d}_{uuid.uuid4().hex[:6]}"
        dst_img  = img_dir / f"{tag}_{src.name}"
        dst_lab  = lab_dir / f"{tag}_{src.stem}.txt"

        os.symlink(src.resolve(), dst_img)      # enlace a la imagen
        os.symlink((TEST_LAB / f"{src.stem}.txt").resolve(), dst_lab)

    yaml_path = set_dir / "data.yaml"
    yaml_path.write_text(yaml.dump({
        "names": CLASS_NAME,
        "nc"   : len(CLASS_NAME),
        "train" : str(img_dir),
        "val" : str(img_dir),
        "test" : str(img_dir)
    }))


In [ ]:
import pathlib, time, pandas as pd
from ultralytics import YOLO

BOOT_SETS = ROOT_PATH / BOOT_ROOT 
OUT_ROOT = ROOT_PATH / pathlib.Path("BOOTSTRAP")     
DEVICE = "mps"                            
BATCH = 64                               


MODELS = {
    "YOLOV11N": "ENTRENAMIENTO_YOLOV11/YOLOV11_AdamW_lr0.001_wd1e-4_mos1.0/weights/best.pt",
    "YOLOV11S": "ENTRENAMIENTO_YOLOV11S/YOLOV11S_AdamW_lr0.0005_wd0.0005_mos0/weights/best.pt",
    "YOLOV11M": "ENTRENAMIENTO_YOLOV10S/YOLOV10S_AdamW_lr0.0005_wd0.0005_mos0/weights/best.pt"
}

yaml_files = sorted(BOOT_SETS.glob("set_*/data.yaml"))
assert yaml_files, "No se encontraron réplicas en BOOTSTRAP/SETS/"

rows = []
for model_name, model_path in MODELS.items():
    print(f"Evaluando {model_name} …")
    model = YOLO(model_path)               

    for yml in yaml_files:
        start = time.time()
        r = model.val(
            data=str(yml),
            split="test",
            batch=BATCH,
            device=DEVICE,
            workers=0,      
            verbose=False,
            save=False,
            plots=False
        )

        rows.append({
            "Model"   : model_name,
            "Sample"  : yml.parent.name,     
            "Precision": r.box.mp,
            "Recall"   : r.box.mr,
            "mAP50"    : r.box.map50,
            "mAP5095"  : r.box.map,
            "Pre_ms"   : r.speed["preprocess"],
            "Inf_ms"   : r.speed["inference"],
            "Post_ms"  : r.speed["postprocess"],
            "Elapsed_s": round(time.time() - start, 2)
        })

    out_dir = OUT_ROOT / model_name
    out_dir.mkdir(exist_ok=True)
    (pd.DataFrame([r for r in rows if r["Model"] == model_name])
        .drop(columns="Model")
        .to_csv(out_dir / "bootstrap_metrics.csv", index=False))

print("CSV listos en BOOTSTRAP/<MODEL>/bootstrap_metrics.csv")

In [ ]:
import pandas as pd
from pathlib import Path

csv_files = {
    "YOLOv11-n": Path("/Users/pepefv97/Downloads/PROYECTO_5/BOOTSTRAP/YOLOV11N/bootstrap_metrics.csv"),
    "YOLOv11-s": Path("/Users/pepefv97/Downloads/PROYECTO_5/BOOTSTRAP/YOLOV11S/bootstrap_metrics.csv"),
    "YOLOv10-s": Path("/Users/pepefv97/Downloads/PROYECTO_5/BOOTSTRAP/YOLOV10S/bootstrap_metrics.csv"),
}

# Métricas numéricas sobre las que quieres intervalos
metric_cols = [
    "Precision", "Recall",
    "mAP50", "mAP5095",
    "Pre_ms", "Inf_ms", "Post_ms"
]

rows = []
for model, path in csv_files.items():
    df = pd.read_csv(path, usecols=metric_cols)
    ci_low  = df.quantile(0.025)
    ci_high = df.quantile(0.975)
    median  = df.median()

    for col in metric_cols:
        rows.append({
            "Model": model,
            "Metric": col,
            "Median": median[col],
            "CI_low": ci_low[col],
            "CI_high": ci_high[col]
        })

summary = pd.DataFrame(rows)

# -------------------------------------------------------------
# 2) Guarda el DataFrame (o imprímelo en pantalla)
# -------------------------------------------------------------
#summary.to_csv("BOOTSTRAP/bootstrap_ci.csv", index=False)
print(summary)


In [ ]:
import pandas as pd
from pathlib import Path

# Rutas a los CSV de réplicas bootstrap (uno por modelo)
csv_files = {
    "YOLOv11-n": Path("/Users/pepefv97/Downloads/PROYECTO_5/BOOTSTRAP/YOLOV11N/bootstrap_metrics.csv"),
    "YOLOv11-s": Path("/Users/pepefv97/Downloads/PROYECTO_5/BOOTSTRAP/YOLOV11S/bootstrap_metrics.csv"),
    "YOLOv10-s": Path("/Users/pepefv97/Downloads/PROYECTO_5/BOOTSTRAP/YOLOV10S/bootstrap_metrics.csv"),
}

rows = []
for model, path in csv_files.items():
    df = pd.read_csv(path)

    # ---------- tiempo total por imagen ----------
    df["Time_ms"] = df["Pre_ms"] + df["Inf_ms"] + df["Post_ms"]

    # Inference únicamente
    inf_q = df["Inf_ms"].quantile([0.025, 0.50, 0.975])
    # Total
    tot_q = df["Time_ms"].quantile([0.025, 0.50, 0.975])

    rows.append({
        "Model": model,
        "Inf_median":  inf_q.loc[0.50],
        "Inf_CI_low":  inf_q.loc[0.025],
        "Inf_CI_high": inf_q.loc[0.975],
        "Total_median":  tot_q.loc[0.50],
        "Total_CI_low":  tot_q.loc[0.025],
        "Total_CI_high": tot_q.loc[0.975],
    })

summary = pd.DataFrame(rows)
#summary.to_csv("BOOTSTRAP/bootstrap_time_ci_extended.csv", index=False)
print(summary)

### Graficas del entrenamiento para comprar modelos

In [ ]:
"""
Gráficas de mAP@0.5 y mAP@0.5:0.95 para tres modelos YOLO
con estilo “journal‑ready”.

Requisitos:
  pip install pandas matplotlib scienceplots
"""
# -*- coding: utf-8 -*-

import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import matplotlib as mpl
import scienceplots

mpl.rcdefaults()
plt.style.use(['science', 'no-latex', 'grid'])  # grid = cuadrícula suave

# 2.  Sobrescribe las fuentes para acentos ---------------------------
mpl.rcParams.update({
    "text.usetex": False,              # sin motor TeX
    "font.family": "sans-serif",
    "font.sans-serif": ["DejaVu Sans"],
    "mathtext.fontset": "dejavusans",  # misma fuente en mathtext
    "mathtext.default": "regular",
    "axes.unicode_minus": False,       # signo menos unicode
    # detalles visuales extra
    "figure.dpi": 300,
    "font.size": 10,
})


# scienceplots ha vuelto a poner mathtext.fontset='cm'.
# Vuelve a SOBRE-ESCRIBIR:
#mpl.rcParams["mathtext.fontset"] = "dejavusans"

# ------------------------------------------------------------------
# 1. Diccionario modelo → ruta CSV (ajusta rutas según tu estructura)
# ------------------------------------------------------------------
csv_paths = {
    "YOLOv11-n": Path("/Users/pepefv97/Downloads/PROYECTO_5/ENTRENAMIENTO_YOLOV11/YOLOV11_AdamW_lr0.001_wd1e-4_mos1.0/results.csv"),
    "YOLOv11-s": Path("/Users/pepefv97/Downloads/PROYECTO_5/ENTRENAMIENTO_YOLOV11S/YOLOV11S_AdamW_lr0.0005_wd0.0005_mos0/results.csv"),
    "YOLOv10-s": Path("/Users/pepefv97/Downloads/PROYECTO_5/ENTRENAMIENTO_YOLOV10S/YOLOV10S_AdamW_lr0.0005_wd0.0005_mos0/results.csv")
}

# ------------------------------------------------------------------
# 2. Cargar solo las columnas necesarias y agregar el nombre del modelo
# ------------------------------------------------------------------
frames = []
for label, path in csv_paths.items():
    df = pd.read_csv(
        path
    )
    df["model"] = label
    frames.append(df)

data = pd.concat(frames, ignore_index=True)




pro_palette = ["#3C5488",  
               "#E64B35",  
               "#00A087"]  

line_styles = ["-", "--", ":"]

In [ ]:
fig, ax = plt.subplots(figsize=(4.8, 3.4))
for i, (label, grp) in enumerate(data.groupby("model")):
        ax.plot(
            grp["epoch"],
            grp["metrics/mAP50(B)"],
            label=label,
            color=pro_palette[i % len(pro_palette)],
            linestyle=line_styles[i % len(line_styles)],
            marker='o',
            markersize=3
        )
ax.set_xlabel("Época")
ax.set_ylabel("mAP@0.5")
ax.set_title("Métrica mAP@0.5 vs Época", pad=6)
ax.legend(frameon=False)
fig.tight_layout()
plt.show() 

fig.savefig("map50_vs_epoca.png", dpi=300, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(figsize=(4.8, 3.4))
for i, (label, grp) in enumerate(data.groupby("model")):
        ax.plot(
            grp["epoch"],
            grp["metrics/mAP50-95(B)"],
            label=label,
            color=pro_palette[i % len(pro_palette)],
            linestyle=line_styles[i % len(line_styles)],
            marker='o',
            markersize=3
        )
ax.set_xlabel("Época")
ax.set_ylabel("mAP@0.5:0.95")
ax.set_title("Métrica mAP@0.5:0.95 vs Época", pad=6)
ax.legend(frameon=False)
fig.tight_layout()
plt.show() 

fig.savefig("map5095_vs_epoca.png", dpi=300, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(figsize=(4.8, 3.4))
for i, (label, grp) in enumerate(data.groupby("model")):
        ax.plot(
            grp["epoch"],
            grp["time"]/60,
            label=label,
            color=pro_palette[i % len(pro_palette)],
            linestyle=line_styles[i % len(line_styles)],
            marker='o',
            markersize=3
        )
ax.set_xlabel("Época")
ax.set_ylabel("Tiempo cumulado ([minutos])")
ax.set_title("Tiempo acumulado de entrenamiento vs Época", pad=6)
ax.legend(frameon=False)
fig.tight_layout()
plt.show() 

fig.savefig("tiempo_vs_epoca.png", dpi=300, bbox_inches="tight")


In [ ]:
# Columna de interés
col_map50 = "metrics/mAP50(B)"

# Índice de la fila con mAP@0.5 máximo dentro de cada modelo
idx_best_per_model = data.groupby("model")[col_map50].idxmax()

# Extraer esas tres filas y (opcional) ordenarlas por mAP@0.5 descendente
best_rows = (
    data.loc[idx_best_per_model]          # filas ganadoras
        .sort_values(col_map50, ascending=False)
        .reset_index(drop=True)
)

print(best_rows)


In [ ]:
data.columns

In [ ]:
data[data["epoch"]==10]

### Parte final

Una vez teniendo nuestras arquitectura, pesos vamos a probar que tan bien funciona nuestro modelo  

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from ultralytics.engine.results import Results
from transformers import pipeline
from PIL import Image
from typing import List, Union, Tuple, Optional
import torch

class ObjectDetector:
    model: YOLO
    detections: Optional[Results]
    raw_img: Optional[np.ndarray]
    crops: List[np.ndarray]
    ocr_results: List[str]
    ocr_pipe: Optional[object]

    def __init__(self, model_name: str) -> None:
        """Carga el modelo YOLO y prepara atributos."""
        self.model = YOLO(model_name)
        self.detections = None
        self.raw_img = None
        self.crops = []
        self.ocr_results = []
        self.ocr_pipe = None

    def predict(self, image: Union[str, np.ndarray]) -> Results:
        """
        Realiza detección y guarda la imagen original en alta resolución en `self.raw_img`.
        """
        # 1) Cargar imagen sin modificar
        if isinstance(image, str):
            bgr = cv2.imread(image)
            if bgr is None:
                raise FileNotFoundError(f"No se pudo leer la imagen: {image}")
            rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
        else:
            bgr = image.copy()
            rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
        self.raw_img = rgb  # guardamos la imagen original :contentReference[oaicite:0]{index=0}

        # 2) Inferencia sobre la misma imagen (YOLO letterbox internamente)
        self.detections = self.model.predict(rgb, verbose=False)[0]
        return self.detections

    def extract_crops(self, keep: float = 0.0) -> List[np.ndarray]:
        """
        Extrae recortes de `self.raw_img` usando las cajas en `self.detections`,
        preservando la calidad original.
        """
        if self.detections is None or self.raw_img is None:
            raise RuntimeError("Ejecute primero predict() antes de extract_crops().")

        self.crops.clear()
    
        for box, conf in zip(
            self.detections.boxes.xyxy.cpu().numpy(),
            self.detections.boxes.conf.cpu().numpy()
        ):
            if conf < keep:
                continue
            x1, y1, x2, y2 = box.astype(int)
            crop = self.raw_img[y1:y2, x1:x2]  # recorte directo de original
            self.crops.append(crop)

        return self.crops

    def init_ocr(self, ocr_model_name: str = "microsoft/trocr-base-printed") -> None:
        """Inicializa el pipeline de OCR detectando el mejor dispositivo."""
        device = "mps" if torch.backends.mps.is_available() \
                 else 0 if torch.cuda.is_available() else -1
        self.ocr_pipe = pipeline("image-to-text", model=ocr_model_name, device=device)

    def ocr_plate(self) -> List[str]:
        """Aplica OCR sobre cada recorte en `self.crops`."""
        if not self.crops:
            raise RuntimeError("Ejecute extract_crops() antes de ocr_plate().")
        if self.ocr_pipe is None:
            self.init_ocr()
        self.ocr_results = [
            self.ocr_pipe(Image.fromarray(c))[0]["generated_text"].strip()
            for c in self.crops
        ]
        return self.ocr_results

    def show_image(self, mode: str = 'detections',
                   figsize: Tuple[float, float] = (8, 6)) -> None:
        """
        Muestra inline (notebook) la imagen con cajas+etiquetas o cada crop,
        usando matplotlib para evitar ventanas externas.
        """
        if mode == 'detections':
            if self.detections is None or self.raw_img is None:
                raise RuntimeError("Ejecute predict() antes de show_image().")
            img = self.raw_img.copy()
            for box, cls_id, conf in zip(
                self.detections.boxes.xyxy.cpu().numpy().astype(int),
                self.detections.boxes.cls.cpu().numpy().astype(int),
                self.detections.boxes.conf.cpu().numpy()
            ):
                x1, y1, x2, y2 = box
                cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)
                label = f"{self.model.names[cls_id]} {conf:.2f}"
                font, fs, th = cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1
                (w, h), _ = cv2.getTextSize(label, font, fs, th)
                ty = y1 - 5 if y1 - 5 > h else y1 + h + 5
                cv2.rectangle(img, (x1, ty-h), (x1+w, ty+2), (0,255,0), cv2.FILLED)
                cv2.putText(img, label, (x1, ty), font, fs, (0,0,0), th, cv2.LINE_AA)
            plt.figure(figsize=figsize)
            plt.imshow(img)
            plt.axis('off')
            plt.show()

        elif mode == 'crops':
            if not self.crops:
                raise RuntimeError("Ejecute extract_crops() antes de show_image().")
            for crop in self.crops:
                plt.figure(figsize=figsize)
                plt.imshow(crop)
                plt.axis('off')
                plt.show()
        else:
            raise ValueError("Modo desconocido. Use 'detections' o 'crops'.")

In [ ]:
det = ObjectDetector('YOLO10S.pt')
results = det.predict('PLACA_PATRULLA.jpg')
det.show_image("detections")    # muestrar detecciones inline
det.extract_crops(keep=0.5)
det.show_image("crops")         # muestrar crops inline
texts = det.ocr_plate()         # aplicar OCR en MPS/GPU/CPU
print(texts)

In [ ]:
results = det.predict('PLACA_PATRULLA.jpg')
det.show_image("detections")    # muestrar detecciones inline
det.extract_crops(keep=0.5)
det.show_image("crops")         # muestrar crops inline
texts = det.ocr_plate() 
print("El texto de la placa es:")# aplicar OCR en MPS/GPU/CPU
print(texts)